In [1]:
import torch, math, time, argparse, os
import random, dataset, utils, losses, net
import numpy as np

from dataset.market import Market
from net.resnet import *
from net.googlenet import *
from net.bn_inception import *
from dataset import sampler
from torch.utils.data.sampler import BatchSampler
from torch.utils.data.dataloader import default_collate

from tqdm import *
import wandb

In [2]:
seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [3]:
LOG_DIR = '.'
trn_dataset = dataset.load('market', '../ITCS-5145-CV/learning/Market-1501-v15.09.15/', 'train', transform = dataset.utils.make_transform(
                                                                                                                        is_train = True, 
                                                                                                                        is_inception = False))

In [4]:
dl_tr = torch.utils.data.DataLoader(
        trn_dataset,
        batch_size = 50,
        shuffle = True,
        num_workers = 4,
        drop_last = True,
        pin_memory = True
    )

In [5]:
eval_dataset = dataset.load('market', '../ITCS-5145-CV/learning/Market-1501-v15.09.15/', 'train', transform = dataset.utils.make_transform(
                                                                                                                        is_train = True, 
                                                                                                                        is_inception = False))

dl_ev = torch.utils.data.DataLoader(
        eval_dataset,
        batch_size = 50,
        shuffle = False,
        num_workers = 4,
        pin_memory = True
)

In [6]:
model_name = 'resnet50'

In [8]:
if model_name == 'resnet50':
    model = Resnet50(embedding_size=512, pretrained=True, is_norm=1, bn_freeze =1).cuda()

/home/nevin/anaconda3/envs/cv/lib/python3.9/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/nevin/anaconda3/envs/cv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
criterion = losses.Proxy_Anchor(nb_classes = trn_dataset.nb_classes(), sz_embed = 512, mrg = 0.1, alpha = 32).cuda()

In [10]:
param_groups = [
    {'params': list(set(model.parameters()).difference(set(model.model.embedding.parameters())))},
    {'params': model.model.embedding.parameters(), 'lr':float(1e-4) * 1},
]
param_groups.append({'params': criterion.parameters(), 'lr':float(1e-4) * 100})

In [11]:
opt = torch.optim.AdamW(param_groups, lr=float(1e-4), weight_decay = 1e-4)

In [12]:
scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=10, gamma = 0.5)

In [13]:
model_dir = 'models'

In [27]:
best_recall=[0]
log = {}
for epoch in range(0, 60):
    model.train()
    bn_freeze = True

    if bn_freeze:
            modules = model.model.modules()
            for m in modules: 
                if isinstance(m, nn.BatchNorm2d):
                    m.eval()

    losses_per_epoch = []
    unfreeze_model_param = list(model.model.embedding.parameters()) + list(criterion.parameters())

    if epoch == 0:
        for param in list(set(model.parameters()).difference(set(unfreeze_model_param))):
            param.requires_grad = False
    if epoch == 1:
        for param in list(set(model.parameters()).difference(set(unfreeze_model_param))):
            param.requires_grad = True

    pbar = tqdm(enumerate(dl_tr))

    for batch_idx, (x, y) in pbar:                         
        m = model(x.squeeze().cuda())
        loss = criterion(m, y.squeeze().cuda())
        
        opt.zero_grad()
        loss.backward()
        
        torch.nn.utils.clip_grad_value_(model.parameters(), 10)
        
        torch.nn.utils.clip_grad_value_(criterion.parameters(), 10)

        losses_per_epoch.append(loss.data.cpu().numpy())
        opt.step()

        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)] Loss: {:.6f}'.format(
                epoch, batch_idx + 1, len(dl_tr),
                100. * batch_idx / len(dl_tr),
                loss.item()))
    scheduler.step()
    
    if(epoch >= 0):
        with torch.no_grad():
            print("**Evaluating...**")
            Recalls = utils.evaluate_cos(model, dl_ev)
        if best_recall[0] < Recalls[0]:
            best_recall = Recalls
            best_epoch = epoch
            if not os.path.exists(model_dir):
                os.makedirs(model_dir)
            torch.save({'model_state_dict':model.state_dict()}, '{}/{}_best.pth'.format(model_dir, model_name))
            with open('{}/{}_best_results.txt'.format(model_dir, model_name), 'w') as f:
                f.write('Best Epoch: {}\n'.format(best_epoch))
                for i in range(6):
                    f.write("Best Recall@{}: {:.4f}\n".format(2**i, best_recall[i] * 100))

Train Epoch: 0 [549/549 (100%)] Loss: 2.359023: : 549it [00:52, 10.52it/s]


**Evaluating...**


100%|█████████████████████████████████████████| 550/550 [00:49<00:00, 11.09it/s]


R@1 : 23.467
R@2 : 29.184
R@4 : 35.552
R@8 : 42.830
R@16 : 50.791
R@32 : 59.568


Train Epoch: 1 [549/549 (100%)] Loss: 1.572577: : 549it [02:29,  3.67it/s]


...

R@1 : 81.929
R@2 : 86.405
R@4 : 90.171
R@8 : 92.908
R@16 : 95.164
R@32 : 96.951


Train Epoch: 59 [549/549 (100%)] Loss: 0.245883: : 549it [02:29,  3.68it/s]


**Evaluating...**


100%|█████████████████████████████████████████| 550/550 [00:49<00:00, 11.06it/s]


R@1 : 81.591
R@2 : 86.089
R@4 : 89.720
R@8 : 92.562
R@16 : 94.782
R@32 : 96.652
